# En İyi Parametreler için ROC Eğrilerinin Oluşturulması

In [8]:
from sklearn.metrics import confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from itertools import cycle

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn import metrics

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from keras import callbacks
import tensorflow as tf
import keras

def create_roc_curve(y_test_encoded, y_pred_encoded, name):

    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    for i in range(len(y_test_encoded[0])):
        fpr[i], tpr[i], _ = roc_curve(
            y_test_encoded[:, i], y_pred_encoded[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    fpr["micro"], tpr["micro"], _ = roc_curve(
        y_test_encoded.ravel(), y_pred_encoded.ravel())

    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    all_fpr = np.unique(np.concatenate(
        [fpr[i] for i in range(len(y_test_encoded[0]))]))
    mean_tpr = np.zeros_like(all_fpr)

    for i in range(len(y_test_encoded[0])):
        mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

    mean_tpr /= len(y_test_encoded[0])
    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])
    plt.figure(figsize=(12, 12))
    plt.plot(fpr["micro"], tpr["micro"],
             label='micro-average ROC curve (area = {0:0.2f})'
             ''.format(roc_auc["micro"]),
             color='deeppink', linestyle=':', linewidth=4)

    plt.plot(fpr["macro"], fpr["macro"],
             label='macro-average ROC curve (area = {0:0.2f})'
             ''.format(roc_auc["macro"]),
             color='navy', linestyle=':', linewidth=4)

    colors = cycle(sns.color_palette())
    for i, color in zip(range(len(y_test_encoded[0])), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=2,
                 label='ROC curve of class {0} (area = {1:0.2f})'
                 ''.format(i, roc_auc[i]))

    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Some extension of Receiver operating characteristic to multi-class')
    plt.legend(loc="lower right")
    plt.savefig('roc_curve_'+name+'.png')
    plt.cla()
    plt.clf()

## Decision Tree için ROC Curve

In [9]:
from sklearn.metrics import confusion_matrix, accuracy_score

df = pd.read_csv('data/clustered_data.csv').drop(['id', 'player_name', 'position'], axis=1)
X = df.drop(['classes'], axis = 1).values
y = df['classes']
n_cols = len(np.unique(y))

sc = StandardScaler()
X = sc.fit_transform(X)

pca = PCA(2)
X = pca.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.36, random_state=14)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
decision_tree = DecisionTreeClassifier(criterion='gini', max_depth=120)

decision_tree.fit(X_train, y_train)

y_pred=decision_tree.predict(X_test) 

create_roc_curve(y_test, y_pred, 'decision_tree')

<Figure size 864x864 with 0 Axes>

## KNN best params için roc eğrisi

In [10]:
from sklearn.metrics import confusion_matrix, accuracy_score

df = pd.read_csv('data/clustered_data.csv').drop(['id', 'player_name', 'position'], axis=1)
X = df.drop(['classes'], axis = 1).values
y = df['classes']
n_cols = len(np.unique(y))

sc = StandardScaler()
X = sc.fit_transform(X)

pca = PCA(2)
X = pca.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.36, random_state=14)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
knn = KNeighborsClassifier(n_neighbors=3)

knn.fit(X_train, y_train)

y_pred=knn.predict(X_test) 

create_roc_curve(y_test, y_pred, 'knn')

<Figure size 864x864 with 0 Axes>

## ANN best params için ROC eğrisi

In [12]:
from sklearn.metrics import confusion_matrix, accuracy_score

df = pd.read_csv('data/clustered_data.csv').drop(['id', 'player_name', 'position'], axis=1)
X = df.drop(['classes'], axis = 1).values
y = df['classes']
n_cols = len(np.unique(y))

sc = StandardScaler()
X = sc.fit_transform(X)

pca = PCA(2)
X = pca.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.36, random_state=14)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
ann = keras.models.load_model('model/model_sc_pca_batch_size_128.h5')


y_pred=ann.predict(X_test) 

create_roc_curve(y_test, y_pred, 'ann')

<Figure size 864x864 with 0 Axes>